In [1]:
import os 
import numpy as np
import pandas as pd
from utils.utils import *
import nibabel as nib

In [2]:
model_path = os.path.join(fits_dir, 'fits_inversed', 'subj01', 'fits_subj01_train_V1_inversed.npy')
model = np.load(model_path, allow_pickle=True)
model.shape

(33797, 9)

In [3]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""


columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)
    for j, roi in enumerate(rois.keys()):
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
       
        scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, j] = scores_per_roi_lh
        results[1, i, j] = scores_per_roi_rh

results[:, 0]
        
        



(2, 8, 12, 12)


array([[[-5.33369627e-02, -3.00514025e-02, -1.55623675e-02,
         -1.96098805e-03, -1.01830697e-02, -1.06628679e-02,
         -2.75733574e-03, -6.53517625e-04,  1.45497287e-04,
         -4.83770146e-03, -1.37285780e-02, -8.21950891e-03],
        [-4.61773747e-02, -2.87481450e-02, -1.47626095e-02,
         -6.76177504e-04, -8.09664730e-03, -1.61620144e-02,
         -1.17117951e-02, -7.53297408e-03, -5.80319045e-03,
         -1.14947283e-02, -2.38376266e-02, -1.52706565e-02],
        [-1.45007432e-02, -1.98025135e-02, -4.11638526e-03,
         -2.88765661e-05, -4.67577414e-03, -1.99108119e-03,
          6.41701772e-04,  2.20469568e-03,  4.40146833e-03,
         -1.51928356e-03, -4.40549560e-03, -1.26900483e-03],
        [-3.89146468e-02, -4.93443139e-02, -3.47223272e-02,
         -7.90498077e-03, -2.14353831e-02, -2.29914200e-02,
         -2.23293489e-02, -1.47182139e-02, -2.43717379e-02,
         -8.92072712e-03, -2.59712260e-02, -2.21453194e-02],
        [-2.00182357e-02, -2.9751856

In [4]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,-0.053337,-0.030051,-0.015562,-0.001961,-0.010183,-0.010663,-0.002757,-0.000654,0.000145,-0.004838,-0.013729,-0.008220,LO-1
V2,-0.046177,-0.028748,-0.014763,-0.000676,-0.008097,-0.016162,-0.011712,-0.007533,-0.005803,-0.011495,-0.023838,-0.015271,hV4
V3,-0.014501,-0.019803,-0.004116,-0.000029,-0.004676,-0.001991,0.000642,0.002205,0.004401,-0.001519,-0.004405,-0.001269,LO-1
hV4,-0.038915,-0.049344,-0.034722,-0.007905,-0.021435,-0.022991,-0.022329,-0.014718,-0.024372,-0.008921,-0.025971,-0.022145,hV4
VO-1,-0.020018,-0.029752,-0.029716,-0.009139,-0.010333,-0.013978,-0.001577,0.003156,-0.004237,-0.002005,-0.000403,0.003008,PHC-2
VO-2,-0.037710,-0.046306,-0.044694,-0.008779,-0.022460,-0.026102,-0.018129,-0.010602,-0.016947,-0.015558,-0.035022,-0.027769,hV4
PHC-1,0.002946,0.000597,0.002243,-0.016933,-0.018165,-0.000614,-0.010867,-0.009141,-0.007481,-0.015665,-0.019176,-0.022155,V1
PHC-2,-0.020499,-0.033146,-0.051437,-0.041019,-0.120086,-0.116596,-0.083062,-0.079099,-0.043037,-0.080316,-0.138359,-0.122041,V1
LO-1,-0.024236,-0.034303,-0.046219,-0.020387,-0.065867,-0.138283,-0.048766,-0.044391,-0.025373,-0.053590,-0.096244,-0.074466,hV4
LO-2,-0.000545,0.006137,0.010308,0.006004,-0.006232,-0.013571,-0.035990,-0.016974,0.001752,-0.022813,-0.035992,-0.013145,V3


In [5]:
subj01_results_rh = pd.DataFrame(results[1, 0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,-0.015240,-0.003936,0.003847,-0.004413,-0.003422,-0.008942,-0.008375,-0.010261,0.016981,-0.000129,-0.015762,-0.007589,LO-1
V2,-0.064659,-0.026772,-0.023106,-0.012964,-0.009844,-0.024387,-0.024398,-0.020479,0.014919,-0.001090,-0.025793,-0.008731,LO-1
V3,-0.008107,-0.020151,0.000232,-0.007845,-0.001405,-0.003824,-0.001337,-0.000409,0.013586,0.001007,-0.012126,-0.002175,LO-1
hV4,-0.035787,-0.037998,-0.022540,-0.014352,-0.014258,-0.028631,-0.029668,-0.026804,0.003179,-0.005408,-0.041668,-0.025678,LO-1
VO-1,-0.003537,-0.006407,-0.005428,-0.011646,-0.002637,-0.004036,-0.002466,-0.004223,0.015955,0.000846,-0.015288,-0.002556,LO-1
VO-2,-0.018038,-0.017563,-0.014537,-0.004754,-0.004677,-0.022307,-0.020912,-0.019290,0.012716,-0.000808,-0.059065,-0.038325,LO-1
PHC-1,0.004416,0.001467,-0.000868,-0.054267,-0.014102,-0.011112,-0.010523,-0.014031,0.009421,-0.003231,-0.053174,-0.041080,LO-1
PHC-2,-0.011411,-0.020845,-0.034996,-0.043394,-0.058832,-0.101091,-0.102793,-0.115064,-0.043183,-0.034879,-0.195013,-0.143505,V1
LO-1,-0.012646,-0.019979,-0.026337,-0.024332,-0.032044,-0.083537,-0.082376,-0.102469,-0.026376,-0.017751,-0.104843,-0.076852,V1
LO-2,0.002316,0.003185,0.004069,0.006416,-0.005444,-0.015752,-0.036026,-0.042748,0.001548,-0.010561,0.020590,0.021733,TO-2


In [6]:
results.shape

(2, 8, 12, 12)

In [69]:
results[0, 0, 0]

array([-6.92103384e-02, -3.85926506e-02, -1.82785452e-02,  4.35458265e-03,
        5.40644238e-04,  1.27054973e-02,  1.29500648e-02,  1.12508767e-02,
        7.44640382e-03, -1.82034736e-06, -1.66749468e-02, -1.25056514e-02])

In [71]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""

columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)

    fits_subj = np.zeros((len(rois.keys()), n_voxels, len(columns)))
    for j, roi in enumerate(rois.keys()):

        



        
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
        
        fits_subj[j] = fits_matrix
       
       # scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
       # scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

      #  results[0, i, j] = scores_per_roi_lh
       # results[1, i, j] = scores_per_roi_rh\
    print(np.mean(fits_subj[0, :cutoff[0],5][belongs[:cutoff[0]] == 'V1']))
    for k, roi in enumerate(rois.keys()):
        scores_per_roi_lh = np.array([np.mean(fits_subj[k, :cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_subj[k, cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, k] = scores_per_roi_lh
        results[1, i, k] = scores_per_roi_rh
    


(2, 8, 12, 12)
-0.05333696268873757
-0.040895265794488704
-0.14563220901157906
-0.05775728105469651
-0.058651624763459684
-0.02925365415950221
-0.06269772730621637
-0.03223483362724862


In [72]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,-0.053337,-0.030051,-0.015562,-0.001961,-0.010183,-0.010663,-0.002757,-0.000654,0.000145,-0.004838,-0.013729,-0.008220,LO-1
V2,-0.046177,-0.028748,-0.014763,-0.000676,-0.008097,-0.016162,-0.011712,-0.007533,-0.005803,-0.011495,-0.023838,-0.015271,hV4
V3,-0.014501,-0.019803,-0.004116,-0.000029,-0.004676,-0.001991,0.000642,0.002205,0.004401,-0.001519,-0.004405,-0.001269,LO-1
hV4,-0.038915,-0.049344,-0.034722,-0.007905,-0.021435,-0.022991,-0.022329,-0.014718,-0.024372,-0.008921,-0.025971,-0.022145,hV4
VO-1,-0.020018,-0.029752,-0.029716,-0.009139,-0.010333,-0.013978,-0.001577,0.003156,-0.004237,-0.002005,-0.000403,0.003008,PHC-2
VO-2,-0.037710,-0.046306,-0.044694,-0.008779,-0.022460,-0.026102,-0.018129,-0.010602,-0.016947,-0.015558,-0.035022,-0.027769,hV4
PHC-1,0.002946,0.000597,0.002243,-0.016933,-0.018165,-0.000614,-0.010867,-0.009141,-0.007481,-0.015665,-0.019176,-0.022155,V1
PHC-2,-0.020499,-0.033146,-0.051437,-0.041019,-0.120086,-0.116596,-0.083062,-0.079099,-0.043037,-0.080316,-0.138359,-0.122041,V1
LO-1,-0.024236,-0.034303,-0.046219,-0.020387,-0.065867,-0.138283,-0.048766,-0.044391,-0.025373,-0.053590,-0.096244,-0.074466,hV4
LO-2,-0.000545,0.006137,0.010308,0.006004,-0.006232,-0.013571,-0.035990,-0.016974,0.001752,-0.022813,-0.035992,-0.013145,V3


In [73]:
subj01_results_rh = pd.DataFrame(results[1, 0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,-0.015240,-0.003936,0.003847,-0.004413,-0.003422,-0.008942,-0.008375,-0.010261,0.016981,-0.000129,-0.015762,-0.007589,LO-1
V2,-0.064659,-0.026772,-0.023106,-0.012964,-0.009844,-0.024387,-0.024398,-0.020479,0.014919,-0.001090,-0.025793,-0.008731,LO-1
V3,-0.008107,-0.020151,0.000232,-0.007845,-0.001405,-0.003824,-0.001337,-0.000409,0.013586,0.001007,-0.012126,-0.002175,LO-1
hV4,-0.035787,-0.037998,-0.022540,-0.014352,-0.014258,-0.028631,-0.029668,-0.026804,0.003179,-0.005408,-0.041668,-0.025678,LO-1
VO-1,-0.003537,-0.006407,-0.005428,-0.011646,-0.002637,-0.004036,-0.002466,-0.004223,0.015955,0.000846,-0.015288,-0.002556,LO-1
VO-2,-0.018038,-0.017563,-0.014537,-0.004754,-0.004677,-0.022307,-0.020912,-0.019290,0.012716,-0.000808,-0.059065,-0.038325,LO-1
PHC-1,0.004416,0.001467,-0.000868,-0.054267,-0.014102,-0.011112,-0.010523,-0.014031,0.009421,-0.003231,-0.053174,-0.041080,LO-1
PHC-2,-0.011411,-0.020845,-0.034996,-0.043394,-0.058832,-0.101091,-0.102793,-0.115064,-0.043183,-0.034879,-0.195013,-0.143505,V1
LO-1,-0.012646,-0.019979,-0.026337,-0.024332,-0.032044,-0.083537,-0.082376,-0.102469,-0.026376,-0.017751,-0.104843,-0.076852,V1
LO-2,0.002316,0.003185,0.004069,0.006416,-0.005444,-0.015752,-0.036026,-0.042748,0.001548,-0.010561,0.020590,0.021733,TO-2
